In [2]:
from influxdb_client import InfluxDBClient
import os

# 🔐 Replace with your actual credentials or load from environment
INFLUX_URL =  "http://localhost:8086"
INFLUX_TOKEN = "pCzyiQ6heX76pm6bcu1crP7xn_bbiTVhGw0LCMpNeJ2bKlbWrF3gATiH1Di5msJ8ANtSZIsvCijCVUXE98-33w=="
INFLUX_ORG = os.getenv("INFLUX_ORG", "Sentry")
INFLUX_BUCKET = os.getenv("INFLUX_BUCKET", "Sentry")

client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
query_api = client.query_api()

# 📦 Query to list all unique symbols from any measurement
flux_query = f'''
import "influxdata/influxdb/schema"

schema.tagValues(
  bucket: "{INFLUX_BUCKET}",
  tag: "symbol"
)
'''

result = query_api.query_data_frame(flux_query)

# ✅ Display symbols
symbols = result["_value"].dropna().unique().tolist()
print("Symbols:", symbols)


Symbols: []


In [6]:
from influxdb_client import InfluxDBClient
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime, timezone

# Setup
url = "http://localhost:8086"
token = "pCzyiQ6heX76pm6bcu1crP7xn_bbiTVhGw0LCMpNeJ2bKlbWrF3gATiH1Di5msJ8ANtSZIsvCijCVUXE98-33w=="
org = "Sentry"
bucket = "US_stocks"
measurement = "candles"

client = InfluxDBClient(url=url, token=token, org=org)

# Define time range - max to delete everything
start = "1970-01-01T00:00:00Z"
stop = datetime.now(timezone.utc).isoformat()

# Define delete predicate
predicate = f'_measurement="{measurement}"'

# Execute delete
delete_api = client.delete_api()
delete_api.delete(start, stop, predicate, bucket=bucket, org=org)

print(f"✅ Measurement '{measurement}' deleted from bucket '{bucket}'")


✅ Measurement 'candles' deleted from bucket 'US_stocks'


In [5]:
fetch_company_name_fmp("AAP")

'Unknown'

In [3]:
from influxdb_client import InfluxDBClient
import json
import os

# 🔐 Replace with your actual credentials or load from environment
INFLUX_URL =  "http://localhost:8086"
INFLUX_TOKEN = os.getenv("INFLUX_TOKEN")
INFLUX_ORG = os.getenv("INFLUX_ORG", "Sentry")
INFLUX_BUCKET = os.getenv("INFLUX_BUCKET", "Sentry")

# client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
# query_api = client.query_api()

def get_all_symbols():
    query = f'''
    import "influxdata/influxdb/schema"
    schema.tagValues(
      bucket: "{INFLUX_BUCKET}",
      tag: "symbol"
    )
    '''
    with InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG) as client:
        query_api = client.query_api()
        result = query_api.query(org=INFLUX_ORG, query=query)
        symbols = [record.get_value() for table in result for record in table.records]
    return symbols

symbols = get_all_symbols()

# Save to JSON file
with open("symbols.json", "w") as f:
    json.dump(symbols, f, indent=2)

print(f"Saved {len(symbols)} symbols to symbols.json")


Saved 0 symbols to symbols.json


In [9]:
import pandas as pd
import os
from collections import Counter

# Load new symbols
df_new = pd.read_csv("symbols.csv")

# Bootstrap from previous learned file
learned_file = "split_symbols_learned.csv"
if os.path.exists(learned_file):
    df_learned = pd.read_csv(learned_file)
    previous_quotes = df_learned['quote_coin'].dropna().unique().tolist()
else:
    df_learned = pd.DataFrame(columns=['symbol', 'base_coin', 'quote_coin'])
    previous_quotes = []

# Initial known quotes
known_quotes = set([
    'USDT', 'BUSD', 'USDC', 'BTC', 'ETH', 'BNB', 'TRY', 'EUR', 'GBP', 'AUD', 'JPY', 'SHIB', 'TRX', 'XRP'
]) | set(previous_quotes)

def split_symbol(symbol, quote_assets):
    for quote in quote_assets:
        if symbol.endswith(quote):
            return symbol[:-len(quote)], quote
    return symbol, None

# First pass: try to split, collect unknown suffixes
unknown_suffixes = []
quote_assets_sorted = sorted(known_quotes, key=len, reverse=True)
base_coins, quote_coins = [], []

for symbol in df_new['symbol']:
    base, quote = split_symbol(symbol, quote_assets_sorted)
    base_coins.append(base)
    quote_coins.append(quote)
    if quote is None:
        # Record all possible suffixes (3-6 chars, customizable)
        for i in range(3, min(7, len(symbol))):
            unknown_suffixes.append(symbol[-i:])

df_new['base_coin'] = base_coins
df_new['quote_coin'] = quote_coins

# Learn new quote coin candidates
suffix_counts = Counter(unknown_suffixes)
learned_quotes = [s for s, count in suffix_counts.items() if count > 1]
quote_assets_sorted = sorted(set(quote_assets_sorted + learned_quotes), key=len, reverse=True)

# Second pass: re-split only those with unknown quote
for idx, row in df_new[df_new['quote_coin'].isnull()].iterrows():
    base, quote = split_symbol(row['symbol'], quote_assets_sorted)
    df_new.at[idx, 'base_coin'] = base
    df_new.at[idx, 'quote_coin'] = quote

# Merge and deduplicate
df_combined = pd.concat([df_learned, df_new]).drop_duplicates(subset='symbol', keep='last')
df_combined.to_csv(learned_file, index=False)

# Report
print("Newly learned quote coins:", learned_quotes)
print(f"Saved {len(df_combined)} total symbols to {learned_file}")


Newly learned quote coins: ['BRL', 'ABRL', 'USD', 'DUSD', 'FDUSD', 'BFDUSD', 'TUSD', 'BTUSD', 'XFDUSD', 'HFDUSD', 'BBRL', 'ARS', 'CBRL', 'TCBRL', 'DAI', 'CFDUSD', 'PLN', 'RON', 'UAH', 'ZAR', 'RFDUSD', 'EFDUSD', 'TBRL', 'TFDUSD', 'LFDUSD', 'PFDUSD', 'IFDUSD']
Saved 770 total symbols to split_symbols_learned.csv


In [4]:
import requests

COMMON_BUZZWORDS = ["ETF", "SEC", "DeFi", "halving", "bullish", "bearish"]

def generate_keywords_for_symbol(symbol):
    # Remove quote currencies
    base_symbol = symbol.replace("USDT", "").replace("BUSD", "").replace("USD", "")

    try:
        url = "https://api.coingecko.com/api/v3/coins/list"
        response = requests.get(url, timeout=10)
        coins = response.json()

        for coin in coins:
            if coin["symbol"].lower() == base_symbol.lower():
                name = coin["name"]
                return [
                    name,
                    base_symbol.upper(),
                    f"{base_symbol.upper()}/USD",
                    f"{name} price",
                    f"{name} news"
                ]
    except Exception as e:
        print(f"⚠️ API Error for {symbol}: {e}")

    # If no match found
    return [
        base_symbol.upper(),
        f"{base_symbol.upper()}/USD",
        "cryptocurrency"
    ]

def enrich_keywords(keywords):
    return keywords + COMMON_BUZZWORDS


In [4]:
from datetime import datetime

end_dt= datetime.now()
# strs_dt
str(end_dt.date())

'2025-05-16'

In [14]:
def fetch_symbols():
    flux = f'''
    import "influxdata/influxdb/schema"
    schema.tagValues(
      bucket: "{INFLUX_BUCKET}",
      tag: "symbol"
    )
    '''
    return(query_api.query_stream(flux))
    for table in query_api.query_stream(flux):
        for rec in table:
          print(rec)
          yield rec.get_value()


In [15]:
s=fetch_symbols()
import pandas as pd

df = query_api.query_data_frame(flux_query)
print(df.head())


c:\Users\srb73\AppData\Local\Programs\Python\Python310\lib\site-packages\influxdb_client\client\warnings.py:31: MissingPivotFunction: The query doesn't contains the pivot() function.

The result will not be shaped to optimal processing by pandas.DataFrame. Use the pivot() function by:

    
import "influxdata/influxdb/schema"

schema.tagValues(
  bucket: "Sentry",
  tag: "symbol"
)
 |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")

You can disable this warning by:
    import warnings
    from influxdb_client.client.warnings import MissingPivotFunction

    warnings.simplefilter("ignore", MissingPivotFunction)

For more info see:
    - https://docs.influxdata.com/resources/videos/pivots-in-flux/
    - https://docs.influxdata.com/flux/latest/stdlib/universe/pivot/
    - https://docs.influxdata.com/flux/latest/stdlib/influxdata/influxdb/schema/fieldsascols/

  warnings.warn(message, MissingPivotFunction)


NameResolutionError: <urllib3.connection.HTTPConnection object at 0x000002110FF6E230>: Failed to resolve 'influxdb' ([Errno 11001] getaddrinfo failed)

In [ ]:
symbols = list(fetch_symbols())
print(symbols)


ApiException: (401)
Reason: Unauthorized
HTTP response headers: HTTPHeaderDict({'Content-Type': 'application/json; charset=utf-8', 'X-Influxdb-Build': 'OSS', 'X-Influxdb-Version': 'v2.7.11', 'X-Platform-Error-Code': 'unauthorized', 'Date': 'Sat, 26 Apr 2025 23:20:04 GMT', 'Content-Length': '55'})
HTTP response body: b'{"code":"unauthorized","message":"unauthorized access"}'


In [18]:
keywords_btc = generate_keywords_for_symbol("ETH")
print(keywords_btc)

# Output:
# ['Bitcoin', 'BTC', 'BTC/USD', 'Bitcoin price', 'Bitcoin news']


['Bifrost Bridged ETH (Bifrost)', 'ETH', 'ETH/USD', 'Bifrost Bridged ETH (Bifrost) price', 'Bifrost Bridged ETH (Bifrost) news']


In [16]:
from influxdb_client import InfluxDBClient

# Replace with your actual credentials
INFLUX_URL = "http://localhost:8086"
INFLUX_TOKEN = "ulx8xRTPRxKpGKTf0Vl_nfioXzS7vunTOxms74HY4tXXHrfPvrgrVrmj48VCOAStqoXNVV3V2b-Bar7SR0ihUw=="
INFLUX_ORG = "Sentry"
INFLUX_BUCKET = "Sentry"

client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)


In [4]:
import asyncio
from datetime import datetime, timedelta, timezone
from alpaca.data.historical import StockHistoricalDataClient
from alpaca.data.requests import StockBarsRequest
from alpaca.data.timeframe import TimeFrame
import os
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

ALPACA_API_KEY = os.getenv('ALPACA_API_KEY')
ALPACA_SECRET_KEY = os.getenv("ALPACA_SECRET_KEY")

stock_data_client = StockHistoricalDataClient(ALPACA_API_KEY, ALPACA_SECRET_KEY)

async def get_bars(symbol: str, start: datetime, end: datetime):
    request = StockBarsRequest(
        symbol_or_symbols=symbol,
        start=start,
        end=end,
        timeframe=TimeFrame.Minute
    )
    bars_response = await asyncio.to_thread(stock_data_client.get_stock_bars, request)
    return bars_response.data.get(symbol, [])

# Set your parameters
symbol = "AAPL"
start_dt = datetime.now(timezone.utc) - timedelta(days=1)
end_dt = datetime.now(timezone.utc) - timedelta(minutes=20)  # at least 15-20 min ago

bars = await get_bars(symbol, start_dt, end_dt)


print(f"Bars type: {type(bars)}")
if bars:
    print(f"First bar: {bars[0]}")
    print(f"Bar attributes: {vars(bars[0])}")
    # Optionally, show all bars as dicts
    bars_dicts = [vars(bar) for bar in bars[:3]]
    print("First 3 bars as dicts:")
    for d in bars_dicts:
        print(d)
else:
    print("No bars returned.")


Bars type: <class 'list'>
First bar: symbol='AAPL' timestamp=datetime.datetime(2025, 5, 22, 22, 2, tzinfo=TzInfo(UTC)) open=201.9 high=201.9 low=201.9 close=201.9 volume=255.0 trade_count=30.0 vwap=201.9
Bar attributes: {'symbol': 'AAPL', 'timestamp': datetime.datetime(2025, 5, 22, 22, 2, tzinfo=TzInfo(UTC)), 'open': 201.9, 'high': 201.9, 'low': 201.9, 'close': 201.9, 'volume': 255.0, 'trade_count': 30.0, 'vwap': 201.9}
First 3 bars as dicts:
{'symbol': 'AAPL', 'timestamp': datetime.datetime(2025, 5, 22, 22, 2, tzinfo=TzInfo(UTC)), 'open': 201.9, 'high': 201.9, 'low': 201.9, 'close': 201.9, 'volume': 255.0, 'trade_count': 30.0, 'vwap': 201.9}
{'symbol': 'AAPL', 'timestamp': datetime.datetime(2025, 5, 22, 22, 3, tzinfo=TzInfo(UTC)), 'open': 201.6994, 'high': 201.9325, 'low': 201.6994, 'close': 201.9325, 'volume': 3227.0, 'trade_count': 36.0, 'vwap': 201.752709}
{'symbol': 'AAPL', 'timestamp': datetime.datetime(2025, 5, 22, 22, 4, tzinfo=TzInfo(UTC)), 'open': 201.72, 'high': 201.72, 'low

In [20]:
import requests
def get_active_binance_symbols():
    url = "https://api.binance.com/api/v3/exchangeInfo"
    response = requests.get(url)
    data = response.json()
    print(data)
    symbols = [
        s['symbol'] for s in data['symbols']
        if s['status'] == 'TRADING'
    ]
    return symbols

In [23]:
# --- Setup ---
import pandas as pd
import json
import os
from influxdb_client import InfluxDBClient, Point
from influxdb_client.client.write_api import SYNCHRONOUS
from datetime import datetime, timezone


# --- InfluxDB Connection ---
# Replace with your actual credentials
INFLUX_URL = "http://localhost:8086"
INFLUX_TOKEN = "VcIF-9WDH2I3Eyhxk-9L_AXRo2Qg1T1vVBgYPYEt6hwJEhfRqcrI101BczDcVBwWQhW3clVzCkg7pmhIFgn7Bg=="
INFLUX_ORG = "Sentry"
INFLUX_BUCKET = "Sentry"

client = InfluxDBClient(url=INFLUX_URL, token=INFLUX_TOKEN, org=INFLUX_ORG)
write_api = client.write_api(write_options=SYNCHRONOUS)
query_api = client.query_api()

# --- Load symbols.csv ---
symbols_df = pd.read_csv("symbols.csv")
symbols = symbols_df["symbol"].dropna().unique().tolist()
print(f"✅ Loaded {len(symbols)} symbols.")

# --- Fixed Buzzwords ---
FIXED_BUZZWORDS = ["ETF", "DeFi", "SEC", "Bullish", "Bearish", "Pump", "Crash", "Rally", "Halving"]

# --- Simple Keyword Generator ---
def generate_keywords(symbol):
    base = symbol.replace("USDT", "").replace("BUSD", "").replace("USD", "")
    return [base.upper(), f"{base.upper()}/USD", f"{base.capitalize()}", f"{base.capitalize()} news"]

def get_existing_symbols():
    flux_query = f'''
    from(bucket: "{INFLUX_BUCKET}")
    |> range(start: -90d)
    |> filter(fn: (r) => r["_measurement"] == "symbol_metadata")
    |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    |> keep(columns: ["symbol"])
    |> group()
    |> distinct(column: "symbol")
    '''
    result = query_api.query_data_frame(flux_query)
    if not result.empty:
        return set(result["symbol"].dropna().unique())
    return set()

existing_symbols = get_existing_symbols()
print(f"✅ {len(existing_symbols)} symbols already exist in Influx.")

# --- Write metadata ---
new_symbols = set(symbols) - existing_symbols
print(f"🔄 Preparing to write {len(new_symbols)} new symbols.")

for symbol in new_symbols:
    base_keywords = generate_keywords_for_symbol(symbol)
    enriched_keywords = enrich_keywords(base_keywords)

    now = datetime.now(timezone.utc)  # <-- Move inside the loop

    point = (
    Point("symbol_metadata")  # <-- explicitly here
    .tag("symbol", symbol)
    .field("keywords", ",".join(enriched_keywords))
    .field("buzzwords", ",".join(COMMON_BUZZWORDS))
    .time(now)
)

    write_api.write(bucket=INFLUX_BUCKET, record=point)
    print(f"✅ Metadata written for {symbol} at {now}")



print("🎯 All new symbols pushed successfully!")


✅ Loaded 770 symbols.
✅ 0 symbols already exist in Influx.
🔄 Preparing to write 770 new symbols.


NameError: name 'generate_keywords_for_symbol' is not defined

In [6]:
def fetch_coin_creation_date(session, coin_id):
    url = f"https://api.coingecko.com/api/v3/coins/{coin_id}"
    try:
        with session.get(url) as resp:
            if resp.status == 200:
                data = resp.json()
                genesis_date_str = data.get("genesis_date")  # e.g. "2013-04-28"
                if genesis_date_str:
                    # Convert to aware datetime at midnight UTC
                    return datetime.strptime(genesis_date_str, "%Y-%m-%d").replace(tzinfo=timezone.utc)
            # fallback or no genesis_date provided
            return None
    except Exception as e:
        print(f"[ERROR] Fetching creation date for {coin_id}: {e}")
        return None

In [ ]:
fetch_coin_creation_date(session, "bitcoin")

RuntimeError: asyncio.run() cannot be called from a running event loop

In [20]:
query = f'''
import "influxdata/influxdb/schema"
from(bucket: "{INFLUX_BUCKET}")
|> range(start: -7d)
|> filter(fn: (r) => r["_measurement"] == "symbol_metadata")
|> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
'''

df = query_api.query_data_frame(query)
print(df)

    result  table                           _start  \
0  _result      0 2025-04-19 21:34:46.276356+00:00   

                             _stop                            _time  \
0 2025-04-26 21:34:46.276356+00:00 2025-04-26 21:32:03.510226+00:00   

      _measurement    symbol buzzwords           keywords  
0  symbol_metadata  TESTCOIN  TestBuzz  Test,Example,Coin  
